# COMP 551 - IMDB

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.text import *

## Load Data

In [3]:
from google.colab import *
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! mkdir -p ~/.kaggle/
! cp 'gdrive/My Drive/Colab Notebooks/kaggle/kaggle.json' ~/.kaggle/

In [5]:
! echo "{ 'data_path': '/content/comp551/imdb/', 'model_path': '/content/comp551/models/' }" > ~/.fastai/config.yml
! mkdir /content/comp551/
path = Config.data_path()
path.mkdir(exist_ok=True)

/bin/bash: /root/.fastai/config.yml: No such file or directory


In [6]:
! kaggle competitions download -c comp-551-imbd-sentiment-classification -p {path}  
! unzip -q -n {path}/train.zip -d {path}
! unzip -q -n {path}/test.zip -d {path}

 45% 9.00M/19.9M [00:00<00:00, 45.6MB/s]
100% 19.9M/19.9M [00:00<00:00, 78.8MB/s]
 89% 25.0M/28.1M [00:00<00:00, 23.9MB/s]
100% 28.1M/28.1M [00:00<00:00, 71.0MB/s]
  0% 0.00/184k [00:00<?, ?B/s]
100% 184k/184k [00:00<00:00, 174MB/s]


## Language model

In [0]:
data_lm = (TextList.from_folder(path=path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train']) 
            .random_split_by_pct(0.1)
            .label_for_lm()           
            .databunch(bs=60))

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [0]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy
1,4.190997,3.988437,0.293246


In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

In [0]:
learn.save_encoder('fine_tuned_enc')

In [0]:
! cp /root/.fastai/data/models/model1.pth /content

In [21]:
! ls /content/fine_tuned_enc.pth /root/.fastai/data/models/

fine_tuned_enc.pth


## Classifier

Now, we'll create a new data object that only grabs the labelled data and keeps those labels. Again, this line takes a bit of time.

In [17]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
#              .split_by_folder(valid='test')
             .random_split_by_pct(0.1)
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=60))

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:487: UserWarning: You are labelling your items with CategoryList.
Your train set contained the following unknown labels, the corresponding items have been discarded.
test
  if getattr(ds, 'warn', False): warn(ds.warn)
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:487: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
test
  if getattr(ds, 'warn', False): warn(ds.warn)


In [25]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')

RuntimeError: ignored

In [0]:
# f_score = partial(fbeta, thresh=0.2, beta = 1)
# learn.metrics = listify([accuracy,f_score])
# learn.metrics = listify([accuracy])

In [0]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy
1,0.531175,0.386603,0.830211


In [0]:
learn.load_encoder('fine_tuned_enc')

In [0]:
learn.save_encoder('model1')

In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy
1,0.405439,0.296256,0.884070


In [0]:
learn.save_encoder('model2')

In [0]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy
1,0.361352,0.268062,0.895825


In [0]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy


RuntimeError: ignored

In [0]:
# learn.predict()